# Test mosaicking methods

In [ ]:
import os
from glob import glob
import rioxarray as rxr
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Define paths to orthoimages, cameras, and reference DEM
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
out_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR_proc_out')
ortho_folder = os.path.join(out_folder, 'final_ortho')
cam_folder = os.path.join(out_folder, 'bundle_adjust')
refdem_file = os.path.join(os.getcwd(), '..', 'inputs', '20251001_Soo_Model_1cm_mean_UTM19N-fake_filled.tif')

# Get images and cameras
image_list = sorted(glob(os.path.join(ortho_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(cam_folder, '*.tsai')))
print(f'Located {len(image_list)} images and {len(cam_list)} cameras')

# Define output folder
mosaic_folder = os.path.join(out_folder, 'mosaic_testing')
os.makedirs(mosaic_folder, exist_ok=True)

## Sample from the closest camera

In [ ]:
# --- Create a grid of the closest camera to each pixel ---
refdem = rxr.open_rasterio(refdem_file).squeeze()
camera_centers = np.array([read_camera_center(f) for f in cam_list])
print('Creating 3D reference grid from DEM')
xv, yv = np.meshgrid(refdem.x.values, refdem.y.values)
Z = refdem.data
xyz_points = np.stack([xv.ravel(), yv.ravel(), Z.ravel()], axis=1)

# Calculate distances to each camera
print('Identifying closest camera to each pixel')
distances = np.linalg.norm(
    xyz_points[:, None, :] - camera_centers[None, :, :],
    axis=2
)
closest_idx = np.argmin(distances, axis=1)
closest_idx_img = closest_idx.reshape(refdem.shape)


def read_camera_center(tsai_path):
    """Parse camera center (C = x y z) from a .tsai pinhole model file."""
    with open(tsai_path) as f:
        cam_lines = f.read().split('\n')
    for line in cam_lines:
        if 'C = ' in line:
            C = line.split(' ')[2:]
            cx_ecef = float(C[0])
            cy_ecef = float(C[1])
            cz_ecef = float(C[2])
    # reproject to UTM
    gdf = gpd.GeoDataFrame(geometry=[Point(cx_ecef, cy_ecef, cz_ecef)], crs="EPSG:4978")
    gdf = gdf.to_crs("EPSG:32619")
    cx, cy, cz = gdf['geometry'].x[0], gdf['geometry'].y[0], gdf['geometry'].z[0]

    return np.array([cx, cy, cz])

def mosaic_from_stack(stack, closest_idx_img_da):
    """Select pixel values from the stack using the nearest-camera index."""
    out_xr = closest_idx_img_da.copy()
    out_xr.data = np.nan * np.ones(closest_idx_img_da.data.shape)
    for i in range(len(stack.camera.data)):
        out_xr = xr.where(closest_idx_img_da==i, stack.isel(camera=i), out_xr)

    return out_xr

datasets = [rxr.open_rasterio(f).squeeze() for f in image_list]
# match refdem grid
datasets = [f.rio.reproject_match(refdem) for f in datasets]
# create stack
stack = xr.concat(datasets, dim="camera")
# set zero values to NaN
stack = xr.where(stack==0, np.nan, stack)

print("Loaded camera centers")

# Create per-pixel 3D coordinates from reference DEM

# convert to DataArray
closest_idx_img_da = xr.DataArray(
    data=closest_idx_img,
    dims=['y', 'x'],
    coords={
        'y': stack.y,
        'x': stack.x
    }
)
# plot
plt.imshow(
    closest_idx_img, cmap='tab20', clim=(0,15),
    extent=(min(refdem.x), max(refdem.x), min(refdem.y), max(refdem.y))
    )
for c in camera_centers:
    plt.plot(c[0], c[1], '*k')
plt.colorbar()
plt.show()

# Create mosaic
print('Creating mosaic')
mosaic_data = mosaic_from_stack(stack, closest_idx_img_da)

# mosaic.rio.write_crs(stack.rio.crs, inplace=True)


In [ ]:
plt.imshow(mosaic_data)
